In [1]:
url = 'https://www.youtube.com/live_chat?is_popout=1&v=' #URL del chat de youtube cuando le das a expandir

In [177]:
import requests
import re
import time
session = requests.Session()
session.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36'
i=0

with open('leidas.txt','w') as file:

    while(True):
        i+=1
        response = session.get(url)
        html = response.text
        #preguntas = re.findall(r'#PREGUNTA(.*?)}', html)
        preguntas = re.findall(r'\"liveChatTextMessageRenderer\":{"message\":{\"simpleText\":(.*?)}', html)
        print(i, end="\r")
        for pregunta in preguntas:
            print(pregunta, file=file)
        

        time.sleep(0.2)
    

KeyboardInterrupt: 

In [ ]:
import requests
import re
import time
session = requests.Session()
session.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36'
response = session.get(url)
html = response.text
#preguntas = re.findall(r'#PREGUNTA(.*?)}', html)
preguntas = re.findall(r'\"liveChatTextMessageRenderer\":{"message\":{\"simpleText\":(.*?)}', html)
preguntas = [w for w in preguntas if '?' in w]
preguntas

In [ ]:
ar = []
from threading import Thread

def threaded_function(ar):
    while True:
        ar.append('test_elem')
        time.sleep(2)
        
thread = Thread(target = threaded_function, args = (ar, ))
thread.start()



In [3]:
from embedding import load_embedding
import numpy as np
emb = load_embedding('/home/iker/Documents/QuestionCluster/TechEmbeddings/embeddings_lower.vec')

In [11]:
emb.word_to_vector('RTX',lower=True)

array([ 0.21721414, -0.02164352,  0.07179935, -0.11760761,  0.06632851,
       -0.00120422, -0.1194934 , -0.1323844 , -0.07958438,  0.19298586,
       -0.17833206, -0.03040629,  0.0307896 , -0.10170139,  0.00680134,
        0.15937783,  0.01616387,  0.09898955,  0.07875012,  0.16374588,
        0.06914695, -0.14290178, -0.0825217 ,  0.0788977 , -0.10443784,
        0.05131805, -0.1096955 ,  0.05479446,  0.04395732,  0.09713656,
        0.00973067,  0.0307978 ,  0.02832987, -0.04009965, -0.06521548,
        0.07115777,  0.03563115, -0.10324487,  0.14166987,  0.07270534,
       -0.06017511,  0.13034898, -0.08137383,  0.06576072,  0.01446174,
        0.12221959, -0.03285577, -0.05983484,  0.08215889,  0.11798887,
        0.0110909 , -0.02774774,  0.12693201,  0.11390778, -0.04941996,
        0.0633297 ,  0.14006284, -0.0209999 , -0.04778424,  0.08657614,
        0.06989306,  0.04481822,  0.06093352, -0.0864757 , -0.05285537,
        0.23822428, -0.17734201,  0.13985378,  0.09953889, -0.07

In [12]:
def calculate_cosine_simil(vector1, vector2):
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

In [16]:
w1 = 'RTX'
w2 = 'i7'
calculate_cosine_simil(emb.word_to_vector(w1,lower=True),emb.word_to_vector(w2,lower=True))

0.40987

In [225]:
import tensorflow as tf
import os 

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    logging.warning("GPU device not found. Without a GPU the execution of the program will be very slow.")
else:
    print('Found GPU at: {}'.format(device_name))


def cosine_knn(ma, mb, k):
    
    X = tf.placeholder(tf.float32, [len(ma), len(ma[0])])
    Y = tf.placeholder(tf.float32, [len(mb), len(mb[0])])
    
    m = tf.matmul(X,tf.transpose(Y))
    _, indexes = tf.nn.top_k(m,k)
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    x = sess.run(indexes, {X:ma, Y:mb})
    sess.close()
    
    return x
    
    


Found GPU at: /device:GPU:0


In [243]:
import collections
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import numpy as np
import logging
from embedding import load_embedding
import re 
import operator
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize



def length_normalize(vectors):
    norms = np.sqrt(np.sum(vectors**2, axis=1))
    norms[norms == 0] = 1
    return vectors / norms[:, np.newaxis]
            

def normalize_questions(question):
    question = question.lower()
    question = re.sub(r"[^a-zA-Z0-9]+", r" ", question)
    question = word_tokenize(question)
    question = [w for w in question if not w in set(stopwords.words('spanish'))] 
    return question

def normalize_vector(vector,norm='l2'):
    vector = np.asarray(vector).reshape(-1,len(vector))
    return normalize(vector,norm=norm)[0]

class question_manager():
    questions=[]
    questions_normalized=[]
    questions_vectors = []
    keywords = collections.defaultdict()
    embedding = None
    
    def __init__(self, embedding_path='/home/iker/Documents/QuestionCluster/TechEmbeddings/embeddings_lower.vec'):
        self.questions=[]
        self.questions_normalized=[]
        self.questions_vectors=[]
        self.keywords = collections.defaultdict()
        self.embedding = load_embedding(embedding_path)
    
    
    def get_keywords(self):
        return sorted(self.keywords.items(), key=operator.itemgetter(1), reverse=True)
    
    def question_to_vector(self,question):
        sentence = np.zeros([self.embedding.dims])
        num_words = 0
        
        for word in question:
            try:
                sentence += self.embedding.word_to_vector(word)
                num_words += 1
            except KeyError as r:
                continue
            
        
        if num_words > 0:
            sentence = sentence/num_words
            return sentence
        else:
            logging.warning('Could not calculate the sentence embedding fot the sentence ' + str(question))
    
    def update_keyword_for_sentence(self, question):
        for word in question:
            try:
                self.keywords[word]+=1
            except KeyError as er:
                self.keywords[word]=1
    
    def print_question(self, question, path='questions.txt'):
        with open(path,'a') as file:
            print(str(question), file = file)
            
    
    def add_question(self,question):
        normalized_question = normalize_questions(question)
        question_vector = self.question_to_vector(normalized_question)
        if len(normalized_question) > 0 and question_vector is not None:

            self.questions.append(question)
            self.questions_normalized.append(normalized_question)
            self.questions_vectors.append(question_vector)
            self.update_keyword_for_sentence(normalized_question)
            self.print_question(question)
    
    
    def load_form_file(self,path):
        with open(path,'r') as file:
            for line in file:
                self.add_question(line)
                
    
    def clustering(self,n_clusters=8):
        kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(np.array(self.questions_vectors))
        cluster_centers = kmeans.cluster_centers_
        labels = kmeans.labels_
        
        
        
        questions_cluster = [[] for x in range(n_clusters)]
        
        
        for i_label, label in enumerate(kmeans.predict(np.array(self.questions_vectors))):
            questions_cluster[label].append(self.questions[i_label])
            
        return questions_cluster
    
    
    def k_nearest(self,sentence, k=1):
        
        vectors_norm = length_normalize(np.array(self.questions_vectors))
        result = cosine_knn([self.question_to_vector(normalize_questions(sentence))], vectors_norm, k = k)
        
        for s in result[0]:
            print(self.questions[s])
        
                
    
        
    
    


In [244]:
del qm
qm = question_manager()

In [245]:
qm.load_form_file('test_questions.txt')
for pregunta in set(preguntas):
    qm.add_question(pregunta)

In [237]:
questions = qm.clustering()

In [249]:
qm.k_nearest('GPU', k=3)

"con una grafica turing podemos retrasmitir con una cpu antigua?"

"¿con una grafica turing podemos retrasmitir con una cpu antigua?"

"como funciona una grafica?"



In [178]:
preguntas = []

with open ('leidas.txt') as file:
    for line in file:
        preguntas.append(line)

In [250]:
len(preguntas)

21598

In [184]:
len()

309

[('ryzen', 55),
 ('mejor', 49),
 ('gtx', 38),
 ('n', 34),
 ('rtx', 34),
 ('procesador', 22),
 ('opinas', 22),
 ('pena', 21),
 ('rx', 21),
 ('i7', 21),
 ('qu', 20),
 ('crees', 20),
 ('intel', 20),
 ('amd', 18),
 ('i5', 17),
 ('si', 16),
 ('2060', 16),
 ('grafica', 16),
 ('1060', 15),
 ('2070', 15),
 ('1070', 15),
 ('recomiendas', 15),
 ('2080', 14),
 ('1660', 14),
 ('3000', 13),
 ('da', 12),
 ('pc', 12),
 ('comprar', 12),
 ('nvidia', 12),
 ('problemas', 12),
 ('1080', 12),
 ('placa', 12),
 ('precio', 12),
 ('puede', 11),
 ('s', 11),
 ('570', 11),
 ('calidad', 11),
 ('usar', 11),
 ('gr', 11),
 ('mas', 10),
 ('hola', 10),
 ('v', 10),
 ('ram', 10),
 ('tener', 10),
 ('5', 10),
 ('vale', 9),
 ('2080ti', 9),
 ('vs', 9),
 ('3', 9),
 ('ser', 9),
 ('buena', 9),
 ('tarjetas', 9),
 ('2', 9),
 ('hacer', 9),
 ('tan', 8),
 ('2700x', 8),
 ('4', 8),
 ('cuello', 8),
 ('iker', 8),
 ('2019', 8),
 ('linux', 8),
 ('tal', 7),
 ('q', 7),
 ('2600', 7),
 ('juegos', 7),
 ('botella', 7),
 ('nuevas', 7),
 ('tarjet

In [202]:
calculate_cosine_simil(qm.question_to_vector('Ryzen vs i7'),qm.question_to_vector('8700K vs 1700X'))

0.9555102125985188

In [200]:
calculate_cosine_simil(qm.question_to_vector('Ryzen vs i7'),qm.question_to_vector('1060 vs 580'))

0.7452799081352657